In [47]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [61]:
df = pd.read_csv('Books_rating.csv')

In [62]:
df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [63]:
df = df[['Title','User_id','review/score']]
df.head()

,Title,User_id,review/score
0,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0
2,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,5.0
3,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0
4,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0


In [64]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Title         object 
 1   User_id       object 
 2   review/score  float64
dtypes: float64(1), object(2)
memory usage: 68.7+ MB
None


In [65]:
print(df.isna().sum())

Title              208
User_id         561787
review/score         0
dtype: int64


In [66]:
df = df.dropna()
df.shape

(2438018, 3)

In [67]:
df.drop_duplicates(subset=['User_id','Title'], keep='first', inplace=True)
df.shape

(2115811, 3)

In [68]:
print(df['Title'].unique().size)
print(df['User_id'].unique().size)

206711
1008961


### Filtering with random books then users with enough ratings

In [109]:
book_rating_counts = df['Title'].value_counts()
books_with_ratings = book_rating_counts[book_rating_counts > 50].index
filtered_df = df[df['Title'].isin(books_with_ratings)]
print(filtered_df.shape)
print(f'users: {filtered_df['User_id'].unique().size}')
print(f'books: {filtered_df['Title'].unique().size}')
print(f'matrix dimensions{filtered_df.shape}')

(1064854, 3)
users: 482370
books: 6223
matrix dimensions(1064854, 3)


In [110]:
#get unique books
unique_books = filtered_df['Title'].unique()

unique_books = unique_books[np.random.randint(len(unique_books),size=1500)]
print(len(unique_books))

filtered_df = filtered_df[filtered_df['Title'].isin(unique_books)]
print(f'users: {filtered_df['User_id'].unique().size}')
print(f'books: {filtered_df['Title'].unique().size}')
print(f'matrix dimensions{filtered_df.shape}')

1500
users: 150930
books: 1342
matrix dimensions(220074, 3)


In [111]:
user_rating_counts = filtered_df['User_id'].value_counts()
users_with_ratings = user_rating_counts[user_rating_counts > 10].index
filtered_df = filtered_df[filtered_df['User_id'].isin(users_with_ratings)]
print(filtered_df.shape)
print(f'users: {filtered_df['User_id'].unique().size}')
print(f'books: {filtered_df['Title'].unique().size}')
print(f'matrix dimensions{filtered_df.shape}')

(19009, 3)
users: 940
books: 1236
matrix dimensions(19009, 3)


In [112]:
df_pivot = filtered_df.pivot_table(index='User_id', columns='Title', values='review/score', aggfunc = 'max')
df_pivot.head()

Title,"'HUMBLE PIE , MY AUTOBIOGRAPHY'","1,000 Places to See Before You Die: A Traveler's Life List",10 Stupid Things Couples Do To Mess Up Their Relationships,1001 Ways to Market Your Books: For Authors and Publishers (Book Marketing Series),1901: A Novel,1906,1st to Die: A Novel,2 Years to a Million in Real Estate,2001: A Space Odyssey,365 Knitting Stitches a Year Perpetual Calendar,...,You Remind Me of Me,Your Body's Many Cries for Water: A Preventive and Self-Education Manual for Those Who Prefer to Adhere to the Logic of the Natural and the Simple in,Your Present: A Half Hour of Peace,Zane's Gettin' Buck Wild: Sex Chronicles II,Zen Shorts (Caldecott Honor Book),Zen in the art of archery,Zig Ziglar's Secrets of Closing the Sale,forever,prince caspian: the return to narnia,the great brain
User_id,,,,,,,,,,,,,,,,,,,,,
A106016KSI0YQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10LWBOIZCF2QT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10T0OW97SFBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
A116J8AUC3JSN2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A11B61QBGHLQDN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df_pivot.to_csv('/home/ldrich/Summer2025BHT/DS_Workflow_Applications/recommender_project/BHTRecommenderProject/ratings_matrix.csv')

In [114]:
filtered_df.to_csv('/home/ldrich/Summer2025BHT/DS_Workflow_Applications/recommender_project/BHTRecommenderProject/filtered_ratings.csv')